# Augmenting LLM with Memory  -  Palm2 & LangChain

## Installation & Authentication

###Install google-generativeai & langchain

In [ ]:
# Install langchain and related libraries
!pip install langchain google-generativeai unstructured

# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform==1.25.0

# Install Gradio for a simple GUI
!pip install -q gradio


###Authenticate

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()



##Define Vertex & LangChain Libraries

###*LangChain Standard Libraries*

In [2]:
# Use LangChain
import langchain
print(f"LangChain version: {langchain.__version__}")

# Use COnversation Buffer Memory & Conversation Chain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms.base import LLM

# Use standard libraries
from pydantic import BaseModel, root_validator
from typing import Any, Mapping, Optional, List, Dict

# Simple Front end libraries
import gradio as gr
import markdown



LangChain version: 0.0.314


###*Use Vertex AI - Google Cloud library*




In [3]:
# Use Vertex AI python SDK
import vertexai

## Initialize Vertex AI

###*Initialize Project & overall platform*

In [5]:
PROJECT_ID = "api-project-503433767370"  # @param {type:"string"}
#PROJECT_ID = "argolis-project-340214"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")
aiplatform.init(project=PROJECT_ID, location=LOCATION)


Vertex AI SDK version: 1.25.0


### Define Vertex Base Class & Additional libraries

In [6]:
class _VertexCommon(BaseModel):
    """Wrapper around Vertex AI large language models.

    To use, you should have the
    ``google.cloud.aiplatform.private_preview.language_models`` python package
    installed.
    """
    client: Any = None #: :meta private:
    model_name: str = "text-bison@001"
    """Model name to use."""

    temperature: float = 0.2
    """What sampling temperature to use."""

    top_p: int = 0.8
    """Total probability mass of tokens to consider at each step."""

    top_k: int = 40
    """The number of highest probability tokens to keep for top-k filtering."""

    max_output_tokens: int = 200
    """The maximum number of tokens to generate in the completion."""

    @property
    def _default_params(self) -> Mapping[str, Any]:
        """Get the default parameters for calling Vertex AI API."""
        return {
            "temperature": self.temperature,
            "top_p": self.top_p,
            "top_k": self.top_k,
            "max_output_tokens": self.max_output_tokens
        }

    def _predict(self, prompt: str, stop: Optional[List[str]]) -> str:
        res = self.client.predict(prompt, **self._default_params)
        return self._enforce_stop_words(res.text, stop)

    def _enforce_stop_words(self, text: str, stop: Optional[List[str]]) -> str:
        if stop:
            return enforce_stop_tokens(text, stop)
        return text

    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "vertex_ai"

class VertexLLM(_VertexCommon, LLM):
    model_name: str = "text-bison@001"

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the python package exists in environment."""
        try:
            from vertexai.preview.language_models import TextGenerationModel
        except ImportError:
            raise ValueError(
                "Could not import Vertex AI LLM python package. "
            )

        try:
            values["client"] = TextGenerationModel.from_pretrained(values["model_name"])
        except AttributeError:
            raise ValueError(
                "Could not set Vertex Text Model client."
            )

        return values

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call out to Vertex AI's create endpoint.

        Args:
            prompt: The prompt to pass into the model.

        Returns:
            The string generated by the model.
        """
        return self._predict(prompt, stop)



In [7]:
# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

llm = VertexLLM(
    model_name='text-bison-32k',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

##Store Chat History into Memory Buffer and pass it to LLM

In [8]:

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory)




## Invoke the LLM to test the memory


In [9]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! I am Sam
AI:

> Finished chain.


' Hello Sam, how may I assist you today? \n'

In [10]:
conversation.predict(input="I live in Dublin, Ohio")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam, how may I assist you today? 

Human: I live in Dublin, Ohio
AI:

> Finished chain.


" Dublin, Ohio is a city in the United States, located in Franklin County, Ohio. It is a suburb of Columbus, Ohio. As of the 2020 census, the city had a population of 49,424. Dublin is home to several large corporations, including Wendy's, Cardinal Health, and Nationwide Insurance. The city is also home to several colleges and universities, including Ohio Wesleyan University and Columbus State Community College. Dublin is known for its excellent schools, safe neighborhoods, and beautiful parks. The city has been ranked as one of the best places to live in the United States by several magazines, including Money magazine and Forbes magazine."

In [11]:
conversation.predict(input="Which is the nearest airport for me to visit")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam, how may I assist you today? 

Human: I live in Dublin, Ohio
AI:  Dublin, Ohio is a city in the United States, located in Franklin County, Ohio. It is a suburb of Columbus, Ohio. As of the 2020 census, the city had a population of 49,424. Dublin is home to several large corporations, including Wendy's, Cardinal Health, and Nationwide Insurance. The city is also home to several colleges and universities, including Ohio Wesleyan University and Columbus State Community College. Dublin is known for its excellent schools, safe neighborhoods, and beautiful parks. The city has been ranked as one of the best places to live in the United

' The nearest airport to Dublin, Ohio, is John Glenn Columbus International Airport (CMH), which is located approximately 15 miles (24 kilometers) southeast of the city. The airport is served by a variety of airlines, including American Airlines, Delta Air Lines, Southwest Airlines, and United Airlines. From John Glenn Columbus International Airport, you can fly to a variety of destinations in the United States and around the world.'

## Build a Front End

In [12]:
newmemory = ConversationBufferMemory()
newconversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=newmemory)

def chatbot(inputtext):
    return newconversation.predict(input=inputtext)

print("Launching Gradio")

iface = gr.Interface(fn=chatbot,
                     inputs=[gr.Textbox(label="Query")],
                     examples=["I live in Plano, Texas", "My flight is at 10PM", "Which is the nearest airport and when shd I start from here to get my flight"],
                     title="Chat With a Bison",
                     outputs=[gr.Textbox(label="Response")],
                     theme=gr.themes.Soft)

iface.launch(share=False)



Launching Gradio


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:611: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>